# Deep Learning Model Experiments with MLflow

**Author:** Uriel G. Rojo (ML Engineer)
**Team:** MLOps 62

In [11]:
# Setup and imports
import sys
import os
import torch

# --- Force CPU for TensorFlow/Keras (do this BEFORE importing keras/tf) ---
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable all GPUs for TF/Keras
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   # silence TF INFO/WARNING logs

# Add project root to path (handles both Docker /work and local environments)
if os.path.exists('/work'):
    sys.path.insert(0, '/work')  # Docker environment
else:
    sys.path.insert(0, os.path.abspath('..'))  # Local environment

import tensorflow as tf
try:
    # Extra safety: make sure TF sees no GPUs at runtime
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MLflow
import mlflow
import mlflow.sklearn
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

mlflow.set_tracking_uri(config.MLFLOW_TRACKING_URI)
mlflow.set_experiment(config.MLFLOW_EXPERIMENT_NAME)

_client = MlflowClient()
_exp = _client.get_experiment_by_name(config.MLFLOW_EXPERIMENT_NAME)
print("MLflow Tracking URI:", mlflow.get_tracking_uri())
print("Experiment:", _exp.name, "| ID:", _exp.experiment_id)

# Custom modules
from src import config
from src.data_utils import load_data
from src.plots import plot_model_performance, plot_feature_importance

# Import Phase 1 feature engine (from features.py file, not features/ directory)
import importlib
features_module = importlib.import_module('src.features')
features_module = importlib.import_module('src.features_engine')
AbsenteeismFeatureEngine = features_module.AbsenteeismFeatureEngine

print("✅ Modules imported successfully")
print(f"MLflow Experiment: {config.MLFLOW_EXPER
    IMENT_NAME}")
from mlflow.models import infer_signature

# --- Diagnostics (confirm CPU mode) ---
# (comments in English only)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("⚠️ GPUs detected but disabled for this run.")
else:
    print("Running on CPU (TF GPUs disabled).")

print(keras.backend.backend())


MLflow Tracking URI: http://mlflow:9001
Experiment: absenteeism-team62 | ID: 2
✅ Modules imported successfully
MLflow Experiment: absenteeism-team62
Running on CPU (TF GPUs disabled).
tensorflow


In [2]:
print(keras.backend.backend())

tensorflow


In [3]:
df_clean = load_data(config.CLEAN_DATA_PATH)
df_clean.head(5)

2025-11-02 21:03:40,493 - src.data_utils - INFO - Detected Docker environment: /work
2025-11-02 21:03:40,494 - src.data_utils - INFO - Repository ready: /work
2025-11-02 21:03:40,496 - src.data_utils - INFO - Reading local file (MD5 verified): /work/data/processed/work_absenteeism_clean_v1.0.csv
2025-11-02 21:03:40,498 - src.data_utils - INFO - Loaded from local: (744, 21)
2025-11-02 21:03:40,499 - src.data_utils - INFO - Data loaded from local: (744, 21)


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11.0,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36.0,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3.0,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7.0,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11.0,23,7,65,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [4]:
monthly_hours = df_clean.groupby(['ID', 'Month of absence']).agg(
    MonthlyAbsenceHours=('Absenteeism time in hours', 'sum')
)
monthly_hours.reset_index()

,ID,Month of absence,MonthlyAbsenceHours
0,1.0,1,1
1,1.0,3,16
2,1.0,4,11
3,1.0,5,8
4,1.0,6,24
...,...,...,...
243,36.0,12,3
244,36.0,224,2
245,110.0,7,8
246,120.0,6,8


In [5]:
monthly_absences = pd.merge(df_clean, monthly_hours, on=['ID', 'Month of absence'], how='inner')
monthly_absences.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,MonthlyAbsenceHours
0,11.0,26,7,3,1,289,36,13,33,239.554,...,1,2,1,0,1,90,172,30,4,70
1,36.0,0,7,3,1,118,13,18,50,239.554,...,1,1,1,0,0,98,178,31,0,9
2,3.0,23,7,4,1,179,51,18,38,239.554,...,1,0,1,0,0,89,170,31,2,48
3,7.0,7,7,5,1,279,5,14,39,239.554,...,1,2,1,1,0,68,168,24,4,6
4,11.0,23,7,65,1,289,36,13,33,239.554,...,1,2,1,0,1,90,172,30,2,70


In [6]:
engine = AbsenteeismFeatureEngine()
df_features = engine.engineer_features(monthly_absences)

print(f"Cleaned data: {df_clean.shape}")
print(f"Feature-engineered data: {df_features.shape}")

# Prepare for modeling
X, y = engine.prepare_for_modeling(df_features, scale_features=True, monthly_model=True)

print(f"\nModel-ready data:")
print(f"  Features (X): {X.shape}")
print(f"  Target (y): {y.shape}")
print(f"  Feature count: {len(engine.feature_names)}")

2025-11-02 21:03:43,718 - src.features_engine - INFO - Starting feature engineering...
2025-11-02 21:03:43,720 - src.features_engine - INFO - Created Absence_Category feature
2025-11-02 21:03:43,721 - src.features_engine - INFO - Created BMI_Category feature
2025-11-02 21:03:43,722 - src.features_engine - INFO - Created Age_Group feature
2025-11-02 21:03:43,723 - src.features_engine - INFO - Created Distance_Category feature
2025-11-02 21:03:43,724 - src.features_engine - INFO - Created Workload_Category feature
2025-11-02 21:03:43,724 - src.features_engine - INFO - Created Season_Name feature
2025-11-02 21:03:43,725 - src.features_engine - INFO - Created High_Risk feature
2025-11-02 21:03:43,725 - src.features_engine - INFO - Feature engineering complete. Created 10 new features
2025-11-02 21:03:43,726 - src.features_engine - INFO - Preparing data for modeling...
2025-11-02 21:03:43,729 - src.features_engine - INFO - Applied feature scaling
2025-11-02 21:03:43,729 - src.features_engin

Cleaned data: (744, 21)
Feature-engineered data: (744, 29)

Model-ready data:
  Features (X): (744, 21)
  Target (y): (744,)
  Feature count: 21


In [7]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=config.TEST_SIZE, 
    random_state=config.RANDOM_STATE
)

print("Data Split:")
print(f"  Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nTarget Statistics:")
print(f"  Train mean: {y_train.mean():.2f}h, std: {y_train.std():.2f}h")
print(f"  Test mean: {y_test.mean():.2f}h, std: {y_test.std():.2f}h")

Data Split:
  Training set: 595 samples (80.0%)
  Test set: 149 samples (20.0%)

Target Statistics:
  Train mean: 58.84h, std: 191.62h
  Test mean: 58.09h, std: 129.49h


In [16]:
# Set MLflow tracking URI
mlflow.set_tracking_uri(config.MLFLOW_TRACKING_URI)

# Ping MLflow (MLflow 3.x)
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

client = MlflowClient()
try:
    # con max_results=1 basta para validar conectividad/autorización
    _ = client.search_experiments(view_type=ViewType.ALL, max_results=1)
    print("✅ MLflow API reachable")
except Exception as e:
    raise RuntimeError(f"❌ Cannot reach server {mlflow.get_tracking_uri()}.\n{e}")

# Create or set experiment
mlflow.set_experiment(config.MLFLOW_EXPERIMENT_NAME)
experiment = client.get_experiment_by_name(config.MLFLOW_EXPERIMENT_NAME)

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment Name: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")

print("\n✅ MLflow configured successfully")
print("UI:", "http://localhost:9001/")

✅ MLflow API reachable
MLflow Tracking URI: http://mlflow:9001
Experiment Name: absenteeism-team62
Experiment ID: 2
Artifact Location: s3://mlopsequipo62/mlops/artifacts/2

✅ MLflow configured successfully
UI: http://localhost:9001/


## Deep Neural Networks for Regression

In [17]:
# Simple Neural Network for Regression

# Note: we force CPU above; do not move the model to any torch device.
# (comments in English only)

num_features = X.shape[1]
model = Sequential()
# model.to(device)  # not applicable for Keras; keep disabled

model.add(Dense(num_features, input_shape=(num_features,), kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(optimizer='adam', loss='mean_squared_error')


In [26]:
num_epochs = 1000
batch_size = 32

# Start MLflow run
with mlflow.start_run(run_name="DL_Regression") as run:
    print(f"MLflow Run ID: {run.info.run_id}")
    
    # Log parameters
    mlflow.log_param("model_type", "keras.models.Model")
    mlflow.log_param("features", len(engine.feature_names))
    mlflow.log_param("train_samples", len(X_train))
    mlflow.log_param("test_samples", len(X_test))
    mlflow.log_param("random_state", config.RANDOM_STATE)
    
    # Train model
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics - Training
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Calculate metrics - Test
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Log metrics
    mlflow.log_metric("train_mae", train_mae)
    mlflow.log_metric("train_rmse", train_rmse)
    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("test_mae", test_mae)
    mlflow.log_metric("test_rmse", test_rmse)
    mlflow.log_metric("test_r2", test_r2)
    
    # Cross-validation
    """
    cv_scores = cross_val_score(lr_model, X_train, y_train, 
                                 cv=5, scoring='neg_mean_absolute_error')
    cv_mae = -cv_scores.mean()
    mlflow.log_metric("cv_mae_mean", cv_mae)
    mlflow.log_metric("cv_mae_std", cv_scores.std())
    """
    # Log model
    mlflow.sklearn.log_model(model, name="model")
    
    # Save feature names
    mlflow.log_dict({"features": engine.feature_names}, "feature_names.json")
    
    print("\n" + "="*60)
    print("NEURAL NETWORK LINEAR REGRESSION RESULTS")
    print("="*60)
    print("\nTraining Metrics:")
    print(f"  MAE:  {train_mae:.3f}")
    print(f"  RMSE: {train_rmse:.3f}")
    print(f"  R2:   {train_r2:.3f}")
    print("\nTest Metrics:")
    print(f"  MAE:  {test_mae:.3f}")
    print(f"  RMSE: {test_rmse:.3f}")
    print(f"  R2:   {test_r2:.3f}")
    # print("\nCross-Validation:")
    # print(f"  CV MAE: {cv_mae:.3f} ± {cv_scores.std():.3f} hours")
    
    # Check target metrics
    print("\nTarget Achievement:")
    print(f"  MAE < 4: {'✅ Yes' if test_mae < 4 else '❌ No'} ({test_mae:.2f})")
    print(f"  RMSE < 8: {'✅ Yes' if test_rmse < 8 else '❌ No'} ({test_rmse:.2f})")
    print(f"  R2 > 0.3: {'✅ Yes' if test_r2 > 0.3 else '❌ No'} ({test_r2:.3f})")

# Visualize performance
# fig = plot_model_performance(y_test, y_test_pred, "Deep NN for Regression")
# plt.show()

# Store results for comparison
lr_results = {
    'model': model,
    'predictions': y_test_pred,
    'test_mae': test_mae,
    'test_rmse': test_rmse,
    'test_r2': test_r2
}

MLflow Run ID: 3713f56329fd41cbac61e62767a7694b
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


2025/11/02 22:41:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



NEURAL NETWORK LINEAR REGRESSION RESULTS

Training Metrics:
  MAE:  27.090
  RMSE: 61.476
  R2:   0.897

Test Metrics:
  MAE:  58.953
  RMSE: 136.231
  R2:   -0.114

Target Achievement:
  MAE < 4: ❌ No (58.95)
  RMSE < 8: ❌ No (136.23)
  R2 > 0.3: ❌ No (-0.114)
🏃 View run DL_Regression at: http://mlflow:9001/#/experiments/2/runs/3713f56329fd41cbac61e62767a7694b
🧪 View experiment at: http://mlflow:9001/#/experiments/2
